In [2]:
# import all files

import pandas as pd

gps1 = pd.read_csv("01.csv", skiprows=5)
gps2 = pd.read_csv("02.csv", skiprows=5)
gps5 = pd.read_csv("05.csv", skiprows=5)
gps7 = pd.read_csv("07.csv", skiprows=5)
gps8 = pd.read_csv("08.csv", skiprows=5)
gps9 = pd.read_csv("09.csv", skiprows=5)
gps10 = pd.read_csv("10.csv", skiprows=5)
gps12 = pd.read_csv("12.csv", skiprows=5)
gps16 = pd.read_csv("16.csv", skiprows=5)

In [3]:
# add name tag to each datafrane

gps1["player"] = "player 01"
gps2["player"] = "player 02"
gps5["player"] = "player 05"
gps7["player"] = "player 07"
gps8["player"] = "player 08"
gps9["player"] = "player 09"
gps10["player"] = "player 10"
gps12["player"] = "player 12"
gps16["player"] = "player 16"

In [4]:
# join everything in one single dataframe

gps = pd.concat([gps1, gps2, gps5, gps7, gps8, gps9, gps10, gps12, gps16], ignore_index=True)

In [5]:
# data cleaning and kpi creation

gps["time[ISO-UTC]"] = pd.to_datetime(gps["time[ISO-UTC]"].str.replace('T', ' '), format='%Y%m%d %H%M%S%f')
gps.drop(columns=["numSatFix[-]", "hdop[-]"], inplace=True)
gps["distance[m]"] = gps["speed[km/h]"]/3.6*0.1
gps['acceleration'] = gps['speed[km/h]'].diff() / (gps['time[ISO-UTC]'].diff().dt.total_seconds()) * (1000 / 3600)

gps = gps[['player', 'time[ISO-UTC]', 'xPos[m]', 'yPos[m]', 'Latitude[deg]', 'Longitude[deg]', 'speed[km/h]', 'hr[bpm]', 'distance[m]', 'acceleration']]

In [6]:
# export csv to use in power bi

# gps.to_csv("gps_full_data.csv")

In [7]:
# function to return sprint kpis

def sprint(df):
    df['is_sprint'] = (df['speed[km/h]'] > 25)
    df['sprint_group'] = (df['is_sprint'] != df['is_sprint'].shift()).cumsum()
    sprints_df = df[df['is_sprint']]
    sprint_stats = sprints_df.groupby('sprint_group').agg(
        Player=('player', 'first'),
        Start_Time=('time[ISO-UTC]', 'first'),
        End_Time=('time[ISO-UTC]', 'last'),
        Start_Latitude=('Latitude[deg]', 'first'),
        Start_Longitude=('Longitude[deg]', 'first'),
        End_Latitude=('Latitude[deg]', 'last'),
        End_Longitude=('Longitude[deg]', 'last'),
        Start_xPos=('xPos[m]', 'first'),
        Start_yPos=('yPos[m]', 'first'),
        End_xPos=('xPos[m]', 'last'),
        End_yPos=('yPos[m]', 'last'),
        Total_Distance=('distance[m]', 'sum'),
        Average_Speed=('speed[km/h]', 'mean'),
        Max_Speed=('speed[km/h]', 'max')
    ).reset_index(drop=True)
    sprint_stats['Sprint Duration'] = (pd.to_datetime(sprint_stats['End_Time']) - pd.to_datetime(sprint_stats['Start_Time'])).dt.total_seconds().round(3)
    sprint_stats.columns = ['Player', 'Start Time', 'End Time', 'Start_Latitude', 'Start_Longitude', 'End_Latitude', 'End_Longitude', 'Start_xPos', 'Start_yPos', 'End_xPos', 'End_yPos', 'Total Distance', 'Average Speed', 'Max Speed', 'Sprint Duration']
    sprint_stats = sprint_stats[['Player', 'Start Time', 'End Time', 'Sprint Duration', 'Start_Latitude', 'Start_Longitude', 'End_Latitude', 'End_Longitude', 'Start_xPos', 'Start_yPos', 'End_xPos', 'End_yPos', 'Total Distance', 'Average Speed', 'Max Speed']]
    return sprint_stats

In [8]:
# export sprint kpis as csv to use in power bi

sprints = sprint(gps)

# sprints.to_csv("sprints_full_data.csv")

In [19]:
# Group by player and calculate KPIs
kpis = gps.groupby('player').agg(
    total_distance=('distance[m]', 'sum'),
    high_intensity_distance=('distance[m]', lambda x: x[gps['speed[km/h]'] > 15].sum()),
    low_intensity_distance=('distance[m]', lambda x: x[gps['speed[km/h]'] <= 15].sum()),
    distance_speed_0_5=('distance[m]', lambda x: x[(gps['speed[km/h]'] >= 0) & (gps['speed[km/h]'] <= 5)].sum()),
    distance_speed_5_15=('distance[m]', lambda x: x[(gps['speed[km/h]'] >= 5) & (gps['speed[km/h]'] <= 15)].sum()),
    distance_speed_15_20=('distance[m]', lambda x: x[(gps['speed[km/h]'] > 15) & (gps['speed[km/h]'] <= 20)].sum()),
    distance_speed_20_25=('distance[m]', lambda x: x[(gps['speed[km/h]'] > 20) & (gps['speed[km/h]'] <= 25)].sum()),
    distance_speed_25_30=('distance[m]', lambda x: x[(gps['speed[km/h]'] > 25) & (gps['speed[km/h]'] <= 30)].sum()),
    distance_speed_30_plus=('distance[m]', lambda x: x[gps['speed[km/h]'] > 30].sum()),
    avg_heart_rate=('hr[bpm]', 'mean'),
    max_heart_rate=('hr[bpm]', 'max'),
    min_heart_rate=('hr[bpm]', 'min'),
    heart_avg_speed_0_5=('hr[bpm]', lambda x: x[(gps['speed[km/h]'] >= 0) & (gps['speed[km/h]'] <= 5)].mean()),
    heart_avg_speed_5_15=('hr[bpm]', lambda x: x[(gps['speed[km/h]'] >= 5) & (gps['speed[km/h]'] <= 15)].mean()),
    heart_avg_speed_15_20=('hr[bpm]', lambda x: x[(gps['speed[km/h]'] > 15) & (gps['speed[km/h]'] <= 20)].mean()),
    heart_avg_speed_20_25=('hr[bpm]', lambda x: x[(gps['speed[km/h]'] > 20) & (gps['speed[km/h]'] <= 25)].mean()),
    heart_avg_speed_25_30=('hr[bpm]', lambda x: x[(gps['speed[km/h]'] > 25) & (gps['speed[km/h]'] <= 30)].mean()),
    heart_avg_speed_30_plus=('hr[bpm]', lambda x: x[gps['speed[km/h]'] > 30].mean()),
    avg_speed=('speed[km/h]', 'mean'),
    max_speed=('speed[km/h]', 'max'),
    min_speed=('speed[km/h]', 'min')
).reset_index()

kpis = kpis.round(2)

# Display the KPIs
kpis

,player,total_distance,high_intensity_distance,low_intensity_distance,distance_speed_0_5,distance_speed_5_15,distance_speed_15_20,distance_speed_20_25,distance_speed_25_30,distance_speed_30_plus,...,min_heart_rate,heart_avg_speed_0_5,heart_avg_speed_5_15,heart_avg_speed_15_20,heart_avg_speed_20_25,heart_avg_speed_25_30,heart_avg_speed_30_plus,avg_speed,max_speed,min_speed
0,player 01,9959.95,1914.75,8045.20,2486.52,5558.68,1443.31,395.30,76.14,0.00,...,86.03,150.40,147.57,144.85,149.97,132.85,NaN,6.02,27.40,0.05
1,player 02,9241.55,860.03,8381.51,2398.82,5982.70,690.91,146.76,22.37,0.00,...,50.07,117.21,120.28,121.32,118.59,118.58,NaN,5.61,26.57,0.06
2,player 05,9260.56,1080.98,8179.58,2739.93,5439.65,864.49,195.69,20.80,0.00,...,50.41,143.81,141.77,141.99,138.26,151.57,NaN,5.59,26.69,0.07
3,player 07,11183.06,1979.28,9203.78,2127.96,7075.82,1570.98,344.36,44.97,18.98,...,51.03,127.85,127.62,128.56,129.51,124.29,106.28,6.76,31.87,0.05
4,player 08,7621.79,1164.52,6457.27,2302.12,4155.15,754.89,310.77,77.16,21.70,...,50.04,151.86,147.14,150.86,146.72,159.54,121.54,5.00,31.94,0.05
5,player 09,10039.39,2231.24,7808.14,2708.95,5099.19,1279.01,773.88,178.35,0.00,...,50.04,133.99,131.22,127.59,126.41,130.05,NaN,6.06,29.73,0.06
6,player 10,11681.14,2428.66,9252.48,1869.15,7383.33,1491.11,659.41,223.53,54.62,...,96.98,162.04,158.41,157.09,154.28,140.42,158.00,7.07,32.09,0.05
7,player 12,10734.27,1635.66,9098.61,2131.58,6967.03,1303.67,321.21,7.32,3.46,...,50.73,131.62,131.14,128.24,133.83,106.94,72.08,6.51,31.96,0.07
8,player 16,4257.21,532.38,3724.83,710.09,3014.74,430.28,102.10,0.00,0.00,...,50.01,101.28,93.14,97.17,96.25,NaN,NaN,6.46,24.54,0.05


In [24]:
kpis[["player", "avg_heart_rate", "heart_avg_speed_0_5"]]

,player,avg_heart_rate,heart_avg_speed_0_5
0,player 01,148.99,150.40
1,player 02,118.63,117.21
2,player 05,142.91,143.81
3,player 07,127.79,127.85
4,player 08,150.16,151.86
5,player 09,132.58,133.99
6,player 10,159.60,162.04
7,player 12,131.23,131.62
8,player 16,96.81,101.28
